In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from juddges.settings import NSA_DATA_PATH
import polars as pl

lf = pl.scan_parquet(NSA_DATA_PATH / "pages" / "pages_chunk_12.parquet")

In [ ]:
df = lf.collect()

In [ ]:
from juddges.data.nsa.extractor import NSADataExtractor, ORDER

extractor = NSADataExtractor()
df = extractor.extract_data_from_pages_to_df(df["page"], df["doc_id"])

In [ ]:
assert len(df.columns) == len(ORDER)

In [ ]:
len(ORDER)

In [ ]:
df.columns

In [ ]:
df[
    df[
        "Published in official collection of judgments Jurisprudence of the Voivodeship Administrative Courts and the Supreme Administrative Court"
    ].notna()
]